In [ ]:
!pip install  DSSATTools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 24.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 56.2 MB/s eta 0:00:00
  Created wheel for fortranformat: filename=fortranformat-2.0.0-py3-none-any.whl size=24536 sha256=47219d95a0f6819fe729e0871a69bd48e0bf15fcf749723a15dc9627f6c96d21
  Stored in directory: /root/.cache/pip/wheels/02/ca/38/f24153dd3ad835514c1cc3fc1d95e461fd120c7d6d308942a8
Successfully built fortranformat


In [ ]:
!pip install meteostat

In [ ]:
import numpy as np
import pandas as pd
import math
from DSSATTools import (
    Crop, SoilProfile, Weather, Management, DSSAT, SoilLayer,
    available_cultivars
)
import DSSATTools
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from meteostat import Point, Daily

In [ ]:
DSSATTools.weather.list_weather_variables()

DATE: Date, year + days from Jan. 1
SRAD: Daily solar radiation, MJ m-2 day-1
TMAX: Daily temperature maximum, C
TMIN: Daily temperature minimum, C
RAIN: Daily rainfall (incl. snow), mm day-1
DEWP: Daily dewpoint temperature average, C
WIND: Daily wind speed (km d-1)
PAR: Daily photosynthetic radiation, moles m-2 day-1
EVAP: Daily pan evaporation (mm d-1)
RHUM: Relative humidity average, %


In [ ]:
latitude=22.5
longitude=88.33
elevation=36

In [ ]:
weather_info=Point(latitude, longitude,elevation)
data=Daily(weather_info, datetime(2022, 1, 1), datetime(2022, 12, 31))
data=data.fetch()

In [ ]:
data

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
time,,,,,,,,,,
2022-01-01,17.9,14.0,23.3,0.0,NaN,343.0,7.4,NaN,1019.3,NaN
2022-01-02,17.5,13.4,23.6,0.0,NaN,340.0,6.8,NaN,1017.7,NaN
2022-01-03,17.3,12.7,23.4,0.0,NaN,342.0,5.6,NaN,1016.5,NaN
2022-01-04,17.0,12.3,23.6,0.0,NaN,357.0,5.7,NaN,1017.1,NaN
2022-01-05,17.5,11.8,24.3,0.0,NaN,345.0,5.5,NaN,1016.4,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-12-27,22.5,18.6,27.4,0.0,NaN,280.0,5.4,NaN,1014.4,NaN
2022-12-28,21.3,17.1,26.2,0.0,NaN,3.0,5.9,NaN,1015.6,NaN
2022-12-29,18.6,13.6,24.6,0.0,NaN,3.0,5.2,NaN,1017.5,NaN


In [ ]:
DATES = pd.date_range('2022-01-01', '2022-12-31')
N = len(DATES)
df = pd.DataFrame(
    {
    'TMIN': data['tmin'].values,
    'TMAX':data['tmax'].values,
    'RAIN': data['prcp'].values,
    'WIND': data['wspd'].values,
    'SRAD': np.random.gamma(10, 1.5, len(DATES))
    },
    index=DATES,
)
df.head()
df=df.fillna(df.mean(),axis=0)

In [ ]:
help(Weather)

Help on class Weather in module DSSATTools.weather:

class Weather(builtins.object)
 |  Weather(df: pandas.core.frame.DataFrame, pars: dict, lat: float, lon: float, elev: float, co2: int = 380)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, df: pandas.core.frame.DataFrame, pars: dict, lat: float, lon: float, elev: float, co2: int = 380)
 |      Initialize a Weather instance. This instance contains the weather data,
 |      as well as the parameters that define the weather station that the data
 |      represents,nsuch as the latitude, longitude and elevation.
 |      
 |      Arguments
 |      ----------
 |      df: DataFrame
 |          pandas DataFrame with the weather data. The index of the dataframe
 |          must be datetime. A simple quality control check is performed for
 |          data. 
 |      pars: dict
 |          A dictionary mapping the data columns to the Weather variables
 |          required by DSSAT. Use `DSSATTools.weather.list_weather_variables` function 

In [ ]:
wth = Weather(
    df,
    {'TMIN': 'TMIN', 'TMAX': 'TMAX', 'RAIN': 'RAIN',
     'WIND':'WIND','SRAD':'SRAD'},
    latitude,longitude,elevation
)
wth

Weather data at 88.330°, 22.500°
  Date start: 2022-01-01
  Date end: 2022-12-31
Average values:
TMIN    22.352877
TMAX    31.269589
RAIN     3.695868
WIND     7.745205
SRAD    14.871163
dtype: float64

In [ ]:
help(SoilProfile)

Help on class SoilProfile in module DSSATTools.soil:

class SoilProfile(builtins.object)
 |  SoilProfile(file: str = None, profile: str = None, default_class: str = None, pars: dict = {})
 |  
 |  Methods defined here:
 |  
 |  __init__(self, file: str = None, profile: str = None, default_class: str = None, pars: dict = {})
 |      Soil Profile class. It can be initialized from an existing file. It also 
 |      can be initialized from scratch.  If a file is provided, then the soil is
 |      initialized as the soil profile with the matching profile id in the file.
 |      
 |      Arguments
 |      ----------
 |      file: str
 |          Optional. Path to the soil file.
 |      profile: str
 |          Optional. Must be passed if file argument is passed. It's the id of 
 |          the profile within the file.
 |      pars: dict
 |          Dict with the non-layer soil parameters. 
 |      default_class: str
 |          Optional. It's a string defining a DSSAT default soil class. If 

In [ ]:
soil = SoilProfile(default_class='CL')
soil

*IBSB910009  IBSNAT      -99     136 Norfolk Sandy Clay Loam (
@SITE        COUNTRY          LAT     LONG SCS FAMILY
 -99         USA            -0.099   -0.099 Fine loamy,silic.,therm. Typic Paleu
@ SCOM  SALB  SLU1  SLDR  SLRO  SLNF  SLPF  SMHB  SMPX  SMKE
   -99  0.14   3.0  0.23  60.0  1.00  0.95 IB001 IB001 IB001
@  SLB  SLMH  SLLL  SDUL  SSAT  SRGF  SSKS  SBDM  SLOC  SLCL  SLSI  SLCF  SLNI  SLHW  SLHB  SCEC  SADC
     5   -99 0.042 0.169 0.392 1.000   -99  0.00  0.00   -99   -99   -99   -99   -99   -99   -99   -99
    15   -99 0.042 0.169 0.392 1.000   -99  0.00  0.00   -99   -99   -99   -99   -99   -99   -99   -99
    25   -99 0.042 0.169 0.392 0.779   -99  0.00  0.00   -99   -99   -99   -99   -99   -99   -99   -99
    33   -99 0.044 0.177 0.358 0.349   -99  0.00  0.00   -99   -99   -99   -99   -99   -99   -99   -99
    46   -99 0.056 0.165 0.396 0.209   -99  0.00  0.00   -99   -99   -99   -99   -99   -99   -99   -99
    61   -99 0.150 0.291 0.377 0.070   -99  0.00  0.00   -99  

In [ ]:
'''
Parameters to be entered after on site evaluation
Example given below
'''

# soilprofile = SoilProfile(
#     pars={
#         'SALB': 0.25, # Albedo
#         'SLU1': 6, # Stage 1 Evaporation (mm)
#         'SLPF': 0.8 # Soil fertility factor
#     }
# )
# layers = [
#     SoilLayer(20, {'SLCL': 50, 'SLSI': 45}),
#     SoilLayer(50, {'SLCL': 30, 'SLSI': 30}),
#     SoilLayer(80, {'SLCL': 32, 'SLSI': 31}),
#     SoilLayer(110, {'SLCL': 30, 'SLSI': 35}),
#     SoilLayer(140, {'SLCL': 24, 'SLSI': 33}),
#     SoilLayer(170, {'SLCL': 20, 'SLSI': 30})
# ]
# for layer in layers: soilprofile.add_layer(layer)
# soilprofile

'\nParameters to be entered after on site evaluation\nExample given below\n'

In [ ]:
available_cultivars("Rice")

['999991',
 '999992',
 '990001',
 '990002',
 '990003',
 '990004',
 'IB0003',
 'IB0012',
 'IB0020',
 'IB0050',
 'IB0055',
 'IB0118',
 'IB0001',
 'IB0002',
 'IB0004',
 'IB0005',
 'IB0006',
 'IB0007',
 'IB0008',
 'IB0009',
 'IB0010',
 'IB0011',
 'IB0013',
 'IB0014',
 'IB0015',
 'IB0016',
 'IB0017',
 'IB0018',
 'IB0019',
 'IB0021',
 'IB0022',
 'IB0023',
 'IB0024',
 'IB0025',
 'IB0026',
 'IB0027',
 'IB0029',
 'IB0030',
 'IB0031',
 'IB0032',
 'IB0115',
 'IB0116',
 'IB0117',
 'IB0119',
 'IB0120',
 'IB0121',
 'IB0122',
 'IB0151',
 'IB0200',
 'IB0051',
 'IB0052',
 'MC0020',
 'TR0001',
 'TR0002',
 'TR0003',
 'TR0004',
 'TR0005']

In [ ]:
help(Crop)

Help on class Crop in module DSSATTools.crop:

class Crop(builtins.object)
 |  Crop(crop_name: str = 'Maize', cultivar_code: str = None)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, crop_name: str = 'Maize', cultivar_code: str = None)
 |      Initializes a Crop instance based on the crop name and the cultivar name.
 |      If the cultivar name is not provided then a custom cultivar will be used. 
 |      
 |      The cultivar and ecotype parameters are represented by the cultivar and 
 |      ecotype attribute, which is a `DSSATTools.sections.Section` instance.
 |      
 |      Arguments
 |      ----------
 |      crop: str
 |          Crop name, available at the moment: Maize, Millet, Sugarbeet, Rice,
 |          Sorghum, Sweetcorn, Alfalfa, Bermudagrass, Soybean, Canola, Sunflower,
 |          Potato, Tomato, Cabbage, Potato and Sugarcane.
 |      cultivar: str
 |          The cultivar identifier. To check the available cultivars for that
 |          crop use the DSSATTools

In [ ]:
crop = Crop("Rice", 'IB0001')

In [ ]:
help(Management)

Help on class Management in module DSSATTools.management:

class Management(builtins.object)
 |  Management(planting_date: datetime.datetime, sim_start: datetime.datetime = None, emergence_date: datetime.datetime = None, initial_swc: float = 0.5, irrigation='N', fertilization='N', harvest='M', organic_matter='G')
 |  
 |  Methods defined here:
 |  
 |  __init__(self, planting_date: datetime.datetime, sim_start: datetime.datetime = None, emergence_date: datetime.datetime = None, initial_swc: float = 0.5, irrigation='N', fertilization='N', harvest='M', organic_matter='G')
 |      Initializes a management instance.
 |      
 |      Arguments
 |      ----------
 |      planting_date: datetime
 |          Planting date.
 |      sim_start: datetime
 |          Date for start of the simulation. If None, it'll be calculated as the
 |          previous day to the planting date.
 |      emergence_date: datetime
 |          Emergence date. If None, I'll be calculated as 5 days after planting.
 | 

In [ ]:
'''
Parameters to be found during on site evaluation
'''

man = Management(
    planting_date=DATES[10],fertilization='R'
)
# Modify irrigation to irrigated
man.simulation_controls['IRRIG'] = "D"

In [ ]:
dssat = DSSAT()
dssat.setup()
dssat.run(
    soil=soil, weather=wth, crop=crop, management=man,
)
output = dssat.output['PlantGro']

/tmp/dssatxceaqlrr created.
Static files copied to /tmp/dssatxceaqlrr.
RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 RI   1  62  97   363   138    32     0    66    58     0   -99      0  157


In [ ]:
'''
Parameters to be found during on site evaluation
'''

man = Management(
    planting_date=DATES[10],fertilization='N'
)
# Modify irrigation to non-irrigated
man.simulation_controls['IRRIG'] = "N"

In [ ]:
crop.cultivar['P1'] = 50
dssat.run(
    soil=soil, weather=wth, crop=crop, management=man,
)
output_2 = dssat.output['PlantGro']

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 RI   1  62  97   363   138    32     0    66    58     0   -99      0  157


In [ ]:
import pickle
with open("/content/rice_future.pkl", "rb") as f:
    rice_prices = pickle.load(f)

In [ ]:
rice_prices

[array([[2206.1577]], dtype=float32),
 array([[2357.1836]], dtype=float32),
 array([[2432.185]], dtype=float32),
 array([[2534.6223]], dtype=float32),
 array([[2680.2983]], dtype=float32)]

In [ ]:
average_price={"wheat": 41463,
               "rice": 39614}
average_cost_of_production={"wheat": 20563,
                            "rice": 19654}

In [ ]:
final_yields=output.CWAD.values
final_yield=final_yields[-1]/1000

In [ ]:
final_yield

1.365

In [ ]:
area=2

In [ ]:
income=area*final_yield*(average_price["rice"]-average_cost_of_production["rice"])
print(income)

54490.8


In [ ]:
amount_requested=10000

In [ ]:
def score(req,income):
  ratio=req/income
  c=50 # Parameter
  return 100*((c**ratio)-1)/((c**ratio)+1)

In [ ]:
final_score=score(amount_requested,income)

In [ ]:
final_score

0.3442992216266966